<a href="https://colab.research.google.com/github/yoo9519-AIdev/Learning/blob/master/%EC%9D%B4%EB%AF%B8%EC%A7%80%EC%B2%98%EB%A6%AC(practice_1D2D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN과 특성 개념  
 - 핵심 특징을 올바르게 추출
 - 다층 신경망과 다층 합성곱 신경망은 각 층별로 저수준 특성을 연결하여, 고수준 특성을 만듦.
 - CNN은 입력 이미지에서 feature map을 만든다.
 - CNN이 이미지 관련 작업을 잘 수행하는 이유는 다음과 같은 2개의 아이디어 때문이다.
  - 희소 연결 : 특성 맵에 있는 하나의 원소는 작은 픽셀 패치 하나에만 연결된다.
  - 파라미터 공유 : 동일한 가중치가 입력 이미지의 모든 패치에 사용된다.  
  즉, 이 두 아이디어의 결과로 네트워크의 가중치(파라미터) 개수가 극적으로 감수하고, 중요 특징을 잡아내는 능력이 향상된다.

## 1D 이산 합성곱 수행
이산 합성곱은 CNN의 기본 연산이다. 텐서플로우 같은 패키지의 실제 합성곱 연산은 훨씬 효율적으로 구성되어 있다.

In [1]:
import numpy as np
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad,
                                   x_padded,
                                   zero_pad])
    res = []
    for i in range(0, int(len(x)/s), s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
        
    return np.array(res)

In [2]:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print("Conv1d 구현 :", conv1d(x, w, p=2, s=1))

Conv1d 구현 : [ 5. 14. 16. 26. 24. 34. 19. 22.]


In [3]:
print("numpy 결과 :", np.convolve(x, w, mode='same'))

numpy 결과 : [ 5 14 16 26 24 34 19 22]


## 2D 합성곱 출력 크기 계산  
합성곱 출력 크기는 입력 벡터 위를 필터 w가 이동하는 전체 횟수로 결정된다. 

In [4]:
import numpy as np
import scipy.signal

In [20]:
def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0] + X_orig.shape[0],
             p[1]:p[1] + X_orig.shape[1]] = X_orig

    res_ = []
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0]) /s[0]) + 1, s[0]):
        res_.append([])
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1]) / s[1]) + 1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0],
                             j:j+W_rot.shape[1]]
            res_[-1].append(np.sum(X_sub * W_rot))
    
    return (np.array(res_))

In [22]:
X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

print("Conv2D 구현:\n", conv2d(X, W, p=(1,1), s=(1, 1)))

Conv2D 구현:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]


In [23]:
print("사이파이 결과:\n", scipy.signal.convolve2d(X, W, mode='same'))

사이파이 결과:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


## 서브샘플링  
전형적인 두 종류의 풀링 연산으로 합성곱 신경망에 적용된다(최대 풀링과 평균 풀링).

In [24]:
pip install imageio